In [1]:
%config Completer.use_jedi = False

import os

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

# example of training an conditional gan on the fashion mnist dataset
from numpy import expand_dims
from numpy import zeros
from numpy import ones
import keras
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Reshape, multiply, Embedding, merge, Concatenate, Conv1D, BatchNormalization
from keras.layers import Dense, Flatten, Multiply
from keras.layers.advanced_activations import LeakyReLU
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
from keras.layers import UpSampling1D
from keras.layers.core import Activation
from keras.layers import Add
import tensorflow as tf
from evaluation_metrics import *
from helper import *
metric_to_calculate = ['FID', 'MMD', 'DTW', 'PC', 'RMSE', 'TWED']

In [2]:
def discriminator(data_dim, input_classes=3):
    
    in_label = Input(shape=(1,))
    x = Embedding(input_classes, 30)(in_label)
    x = Dense(data_dim)(x)
    x = Reshape((data_dim,1))(x)
    
    D_in = Input(shape=[data_dim,1])
    inp1 = Concatenate()([D_in, x])

    x = Conv1D(filters=48, kernel_size=19, padding='same', strides=4, kernel_initializer='he_normal')(inp1)
    # x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=64, kernel_size=15, padding='same', strides=3, kernel_initializer='he_normal')(x)
    # x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=80, kernel_size=11, padding='same', strides=2, kernel_initializer='he_normal')(x)
    # x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=96, kernel_size=9, padding='same', strides=2, kernel_initializer='he_normal')(x)
    # x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=112, kernel_size=7, padding='same', strides=2, kernel_initializer='he_normal')(x)
    # x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = GlobalAveragePooling1D()(x)

    x1 = Conv1D(filters=48, kernel_size=9, padding='same', strides=4, kernel_initializer='he_normal')(inp1)
    # x1 = BatchNormalization()(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = Conv1D(filters=64, kernel_size=7, padding='same', strides=3, kernel_initializer='he_normal')(x1)
    # x1 = BatchNormalization()(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = Conv1D(filters=80, kernel_size=5, padding='same', strides=2, kernel_initializer='he_normal')(x1)
    # x1 = BatchNormalization()(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = Conv1D(filters=96, kernel_size=3, padding='same', strides=2, kernel_initializer='he_normal')(x1)
    # x1 = BatchNormalization()(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = Conv1D(filters=112, kernel_size=3, padding='same', strides=2, kernel_initializer='he_normal')(x1)
    # x1 = BatchNormalization()(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = GlobalAveragePooling1D()(x1)

    xx = concatenate([x,x1])

    xx = Dense(100)(xx)
    xx = Dense(100)(xx)

    out = Dense(1, activation='sigmoid')(xx)

    model = Model(inputs=[D_in, in_label], outputs=out)
    opt = Adam(lr=0.0002, beta_1=0.5)
    loss = 'binary_crossentropy'
    model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])
    return model
    # model.summary()

# d_model = discriminator(data_dim=186, input_classes=3)
# plot_model(d_model, to_file='disc.png', show_shapes=True)

In [3]:
def generator(noise_dim=186, input_classes=3, out_dim=186):
    
    in_label = Input(shape=(1,))
    x = Embedding(input_classes, 30)(in_label)
    x = Dense(noise_dim)(x)
    x = Reshape((noise_dim,1))(x)
    
    G_in = Input(shape=[noise_dim,])
    gen = Dense(noise_dim)(G_in)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((noise_dim,1))(gen)

    x = Concatenate()([gen, x])
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*16, kernel_size=15, strides=1, padding='valid', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
#     x = Activation('relu')(x)

    x = UpSampling1D()(x)
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
#     x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
#     x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
#     x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
#     x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*2, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
#     x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
#     x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=1, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    out = Activation('tanh')(x)
    model = Model(inputs=[G_in, in_label], outputs=out)

    return model

# g_model = generator(noise_dim=100, input_classes=3, out_dim=186)
# plot_model(g_model, to_file='gen.png', show_shapes=True)

In [4]:
def create_gan(d_model, g_model):
    
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise, gen_beat = g_model.input
    # get image output from the generator model
    gen_output = g_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = d_model([gen_output, gen_beat])
    # define gan model as taking noise and label and outputting a classification
    model = Model([gen_noise, gen_beat], gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    loss = 'binary_crossentropy'
    model.compile(loss=loss, optimizer=opt)
    return model

# gan_model = create_gan(d_model, g_model)
# plot_model(gan_model, to_file='gan.png', show_shapes=True)

In [5]:
def reshape(X):
    if len(X.shape) == 1:
        X = X.reshape(X.shape[0], 1)
        return X
    else:
        if X.shape[-1] == 1:
            return X
        else:
            X = X.reshape(X.shape[0], X.shape[1], 1)
            return X

In [6]:
def load_real_samples():
    X = np.load('Data/ForGAN/X.npy')
    y = np.load('Data/ForGAN/y.npy')

    # print (X.shape, y.shape)

    X_N = X[y==0]
    X_S = X[y==1]
    X_V = X[y==2]

    y_N = y[y==0]
    y_S = y[y==1]
    y_V = y[y==2]

    # print (X_N.shape, y_N.shape)
    # print (X_S.shape, y_S.shape)
    # print (X_V.shape, y_V.shape)

#     X_N=X_N.reshape(X_N.shape[0],X_N.shape[1],1)
#     X_S=X_S.reshape(X_S.shape[0],X_S.shape[1],1)
#     X_V=X_V.reshape(X_V.shape[0],X_V.shape[1],1)

    # print (X_N.shape, y_N.shape)
    # print (X_S.shape, y_S.shape)
    # print (X_V.shape, y_V.shape)
    return reshape(X_N), y_N, reshape(X_S), y_S, reshape(X_V), y_V

In [7]:
def generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, n_samples):
    
    # choose random instances
    i_N = randint(0, y_N.shape[0], int(n_samples/3))
    i_S = randint(0, y_S.shape[0], int(n_samples/3))
    i_V = randint(0, y_V.shape[0], int(n_samples/3))
    
    # select ECG and labels
    X = np.vstack((X_N[i_N], X_S[i_S], X_V[i_V]))
    labels = np.hstack((y_N[i_N], y_S[i_S], y_V[i_V]))
    # print (labels.shape)
    
    # generate class labels
    y = reshape(np.random.uniform(0.8, 1, n_samples))
#     y = y.reshape(y.shape[0], 1)
#     y = np.ones((n_samples, 1))
    return [X, labels], y

In [8]:
# generate points in latent space as input for the generator
# normal noise
def generate_latent_points(latent_dim, n_samples, n_classes=3):
    # generate points in the latent space
#     X_fake = np.random.uniform(0, 1.0, size=[n_samples, latent_dim])
    X_fake = np.random.normal(0,1.0,(n_samples,latent_dim))
    # generate labels
    labels_fake = np.hstack((np.zeros(int(n_samples/3)), np.ones(int(n_samples/3)), 2*np.ones(int(n_samples/3))))
    np.random.shuffle(labels_fake)
    return [reshape(X_fake), labels_fake]

In [9]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    ecgs = generator.predict([z_input, labels_input])
    # create class labels
    y = reshape(np.random.uniform(0, 0.2, n_samples))
#     y = y.reshape(y.shape[0], 1)
#     y = np.zeros((n_samples, 1))
    return [ecgs, labels_input], y

In [10]:
# create and save a plot of generated images
def save_plot(X, n):
    plt.figure(figsize=(10,3))
    for i in range(n * n):
        # define subplot
        plt.subplot(n, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.plot(X[i, :, 0])
    plt.show()
    plt.close()

In [11]:
def get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V):
    
    # choose random instances
    i_N = randint(0, y_N.shape[0], 1)
    i_S = randint(0, y_S.shape[0], 1)
    i_V = randint(0, y_V.shape[0], 1)
    
    # select ECG and labels
    X = np.vstack((X_N[i_N], X_S[i_S], X_V[i_V]))
    return X

In [12]:
def save_new_plot(X_R, z_input, n_batch, name):
    n = 3
    Win = (n_batch//3)
    XX = np.vstack((X_R, z_input[0:n,:,:], z_input[Win:Win+n,:,:], z_input[2*Win:2*Win+n,:,:]))
    plt.figure(figsize=(15,5))
    for i in range(n):
        # subplot(R, C, Plot_No)
        plt.subplot(n+1, n, 1 + i)
        plt.axis('off')
        plt.plot(XX[i,:,:])
    for i in range(n, ((n+1)*(n+1))-(n+1)):
        # define subplot
        plt.subplot(n+1, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.plot(XX[i,:,:])
    # plt.show()
    plt.savefig(name, dpi=75)
    plt.close()

In [13]:
# size of the latent space
latent_dim = 100
# size of the data
data = 186
# classes
classes = 3

n_epochs=10

# multiples of three (three classes) (less thyan 24000)
n_batch=300

# Loss Values
G_L = np.infty

# create the discriminator
d_model = discriminator(data_dim=data, input_classes=classes)
# d_model = discriminator(data_dim=data, input_classes=classes)

# create the generator
g_model = generator(noise_dim=latent_dim, input_classes=classes, out_dim=data)
# g_model = generator(noise_dim=latent_dim, input_classes=classes, out_dim=data)

# create the gan
gan_model = create_gan(d_model, g_model)

folder_name = 'Final_CGAN_Beat_Label_Input_Working/'
if not os.path.isdir(folder_name):
    os.mkdir(folder_name)

plot_model(d_model, to_file=folder_name+'disc.pdf', show_shapes=True)
plot_model(g_model, to_file=folder_name+'gen.pdf', show_shapes=True)
plot_model(gan_model, to_file=folder_name+'gan.pdf', show_shapes=True)


# load image data
X_N, y_N, X_S, y_S, X_V, y_V = load_real_samples()
# # train model
# train(g_model, d_model, gan_model, dataset, latent_dim)

bat_per_epo = int(y_S.shape[0] / n_batch)
half_batch = int(n_batch / 2)

plt.ioff()


    
filename = folder_name + 'Plots'
if not os.path.isdir(filename):
    os.mkdir(filename)

model_name = folder_name + 'Model/'
if not os.path.isdir(model_name):
    os.mkdir(model_name)

f = open(folder_name + 'Loss.csv', 'w')
f.write('d_loss1, d_loss2, g_loss \n')
f.close()

f = open(folder_name + 'Stats.csv', 'w')
for i in range(3):
    for mtc in metric_to_calculate:
        f.write(str(mtc)+'_'+str(i)+',')
f.write('\n')
f.close()

# manually enumerate epochs
for i in range(n_epochs):
    # enumerate batches over the training set
    for j in range(bat_per_epo):
        
        # get randomly selected 'real' samples
        [X_real, labels_real], y_real = generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, half_batch)
        # print (X_real.shape, labels_real.shape, y_real.shape)
        # update discriminator model weights
        d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
        
        # generate 'fake' examples
        [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        # print (X_fake.shape, labels.shape, y_fake.shape)
        # update discriminator model weights
        d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
        
        # prepare points in latent space as input for the generator
        [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
        [X, _], _ = generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, n_batch)
        # print (z_input.shape)
        # create inverted labels for the fake samples
        y_gan = reshape(np.random.uniform(0.8, 1, n_batch))
        # update the generator via the discriminator's error
        g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
        
        if g_loss < G_L:
            G_L = g_loss
            g_model.save(model_name + str(i*1000 + j) + '_cgan_generator.h5')

        f = open(folder_name + 'Loss.csv', 'a')
        f.write(str(d_loss1)+','+str(d_loss2)+','+str(g_loss)+'\n')
        f.close()

        if (j+1)%2 == 0:
            print('>%d, %d/%d, d1=%.5f, d2=%.5f g=%.5f' %(i, j, bat_per_epo, d_loss1, d_loss2, g_loss))
            name = filename+'/'+str(i*1000 + j)+'.jpg'
            # generate ECGs
            z_input  = g_model.predict([z_input, labels_input])
            X_R = get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V)
            save_new_plot(X_R, z_input, n_batch, name)
            
#         if (j+1)%10 == 0:
#             evaluate(X, z_input, classes, metric_to_calculate, n_batch, folder_name, samples=4)

>0, 1/80, d1=0.51439, d2=1.35248 g=0.47764
>0, 3/80, d1=0.65761, d2=0.70606 g=0.89714
>0, 5/80, d1=0.84415, d2=0.51116 g=1.08220
>0, 7/80, d1=0.82986, d2=0.50367 g=1.08233
>0, 9/80, d1=0.81716, d2=0.51348 g=1.06272
>0, 11/80, d1=0.79735, d2=0.53053 g=1.01836
>0, 13/80, d1=0.77870, d2=0.56457 g=0.93675
>0, 15/80, d1=0.69041, d2=0.61789 g=0.79646
>0, 17/80, d1=0.59800, d2=0.68967 g=0.72065
>0, 19/80, d1=0.55423, d2=0.73673 g=0.74602
>0, 21/80, d1=0.61772, d2=0.67413 g=0.80779
>0, 23/80, d1=0.61229, d2=0.66571 g=0.76398
>0, 25/80, d1=0.57763, d2=0.68278 g=0.73847
>0, 27/80, d1=0.52905, d2=0.70483 g=0.73430
>0, 29/80, d1=0.54395, d2=0.71421 g=0.74965
>0, 31/80, d1=0.59143, d2=0.70988 g=0.75751
>0, 33/80, d1=0.55747, d2=0.68507 g=0.78209
>0, 35/80, d1=0.55022, d2=0.72511 g=0.74933
>0, 37/80, d1=0.58279, d2=0.69067 g=0.78552
>0, 39/80, d1=0.57315, d2=0.68349 g=0.77362
>0, 41/80, d1=0.58071, d2=0.69382 g=0.78377
>0, 43/80, d1=0.58036, d2=0.75277 g=0.77408
>0, 45/80, d1=0.61348, d2=0.70991 g=0

>4, 55/80, d1=0.62328, d2=0.60872 g=0.91688
>4, 57/80, d1=0.67425, d2=0.62658 g=0.90718
>4, 59/80, d1=0.66442, d2=0.61028 g=0.91985
>4, 61/80, d1=0.65226, d2=0.61059 g=0.91691
>4, 63/80, d1=0.66999, d2=0.62667 g=0.91130
>4, 65/80, d1=0.65530, d2=0.58897 g=0.95111
>4, 67/80, d1=0.65425, d2=0.64496 g=0.88102
>4, 69/80, d1=0.68013, d2=0.62804 g=0.94976
>4, 71/80, d1=0.69563, d2=0.60371 g=0.94928
>4, 73/80, d1=0.66861, d2=0.58711 g=0.96341
>4, 75/80, d1=0.65983, d2=0.64168 g=0.87078
>4, 77/80, d1=0.67726, d2=0.61072 g=0.94313
>4, 79/80, d1=0.64097, d2=0.59994 g=0.97015
>5, 1/80, d1=0.69671, d2=0.63706 g=0.92671
>5, 3/80, d1=0.65675, d2=0.58276 g=0.95473
>5, 5/80, d1=0.63395, d2=0.67427 g=0.84043
>5, 7/80, d1=0.65027, d2=0.58861 g=0.94091
>5, 9/80, d1=0.65863, d2=0.62710 g=0.89134
>5, 11/80, d1=0.67281, d2=0.60751 g=0.97456
>5, 13/80, d1=0.68018, d2=0.61401 g=0.90137
>5, 15/80, d1=0.67717, d2=0.61111 g=0.95788
>5, 17/80, d1=0.67091, d2=0.58839 g=0.97293
>5, 19/80, d1=0.69976, d2=0.58183 g=0

>9, 29/80, d1=0.57058, d2=0.60082 g=1.03519
>9, 31/80, d1=0.59276, d2=0.56773 g=1.09799
>9, 33/80, d1=0.72043, d2=0.58298 g=1.03624
>9, 35/80, d1=0.61547, d2=0.52907 g=1.10154
>9, 37/80, d1=0.64239, d2=0.59162 g=0.96762
>9, 39/80, d1=0.58815, d2=0.57756 g=1.04566
>9, 41/80, d1=0.61992, d2=0.61494 g=0.99443
>9, 43/80, d1=0.62414, d2=0.57804 g=1.10081
>9, 45/80, d1=0.64665, d2=0.55888 g=1.02679
>9, 47/80, d1=0.56744, d2=0.64976 g=1.06552
>9, 49/80, d1=0.64502, d2=0.65229 g=0.98186
>9, 51/80, d1=0.65721, d2=0.56441 g=1.04757
>9, 53/80, d1=0.66530, d2=0.57453 g=1.00242
>9, 55/80, d1=0.64697, d2=0.59176 g=1.02734
>9, 57/80, d1=0.71507, d2=0.56908 g=1.00631
>9, 59/80, d1=0.66305, d2=0.57565 g=0.96522
>9, 61/80, d1=0.60580, d2=0.59551 g=0.97146
>9, 63/80, d1=0.68191, d2=0.60929 g=0.96899
>9, 65/80, d1=0.67493, d2=0.63666 g=1.00297
>9, 67/80, d1=0.67612, d2=0.58428 g=1.00954
>9, 69/80, d1=0.68064, d2=0.61637 g=0.91228
>9, 71/80, d1=0.67275, d2=0.59966 g=0.94729
>9, 73/80, d1=0.65328, d2=0.6174

In [14]:
# name = filename + '/'+str(i*1000 + j)+'.jpg'
# # generate images
# latent_points, labels = generate_latent_points(latent_dim, n*n)
# # specify labels
# [X, labels], y = generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, n*n)
# # generate images
# X  = g_model.predict([latent_points, X])
# X_R = get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V)
# X = np.vstack((X_R, X))
# save_new_plot(X, n+1, name)

In [15]:
# f = open(filename+'_Stats.csv', 'a')
# for k,metric in enumerate(label_dict.keys()):    
#     temp_x = test_data[200*(k):200*(k+1),:-1]
#     [z_input, labels_input] = generate_class_specific_latent_input(200, n_classes=n_classes, noise_dim=noise_dim, category=float(metric))
#     z_input = G.predict([z_input, labels_input], verbose=verbose)

#     for j in range(2):
#         plt.plot(z_input[j])
#     plt.savefig(data_dir+str(i)+'_Label_'+str(metric)+'.png')
#     plt.close()
#     plt.clf()

#     results = evaluate(temp_x,z_input,metric_to_calculate)
#     for r in results:
#         f.write(str(r)+',')

# f.write('\n')
# f.close()

In [16]:
# def save_model(model, data_dir, type='G', epoch=1):
#     json_name = data_dir+str(epoch)+'_'+type+'.json'
#     h5name = data_dir+str(epoch)+'_'+type+'.h5'
#     # serialize model to JSON
#     model_json = model.to_json()
#     with open(json_name, "w") as json_file:
#         json_file.write(model_json)
#     # serialize weights to HDF5
#     model.save_weights(h5name)
#     # print("Saved model to disk")
#     del model


In [17]:
# save_model(model=G, data_dir=data_dir, type='G', epoch=i)
# #     # save the generator model
#     g_model.save('cgan_generator.h5')

# callback = [EarlyStopping(monitor='val_AUC', mode='max', verbose=1, patience=Pat),
#          ModelCheckpoint(filepath=str(twelve_lead_model_filename)+'_check_model.h5', 
#                          monitor='val_AUC', verbose=1, save_best_only=True, mode='max'),
#          ReduceLROnPlateau(monitor='val_AUC', factor=0.5, patience=Pat//2, verbose=1, 
#                            mode='max', min_delta=0.0001, cooldown=0, min_lr=0)]

# # model = parallel_NN(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS):
# model = parallel_NN(Window, len(leads), snomed_classes.shape[0])

# model.compile(loss=tf.keras.losses.BinaryCrossentropy(), 
#               optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
#               metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy', dtype=None, threshold=0.5),
#                        tf.keras.metrics.Recall(name='Recall'),
#                        tf.keras.metrics.Precision(name='Precision'),
#                        tf.keras.metrics.AUC(num_thresholds=200,summation_method="interpolation",
#                                             name="AUC",dtype=None,curve="ROC",thresholds=None,
#                                             multi_label=True,label_weights=None)])
# history = model.fit(train_generator, steps_per_epoch=train_samples, epochs=EP, verbose=1,
#                 validation_data=val_generator, validation_steps=val_samples, callbacks=callback)


# history_name = output_directory + '/' + twelve_lead_filename
# print (twelve_lead_model_filename, history_name)

# save_model(twelve_lead_model_filename, model)
# write_history(history_name, history.history)